# Import dependencies


In [1]:
import pandas as pd
import os
import urllib.request
import json
from dotenv import load_dotenv
from io import StringIO
from bs4 import BeautifulSoup

In [2]:
load_dotenv()

True

# Load stock data


In [3]:
stock_list = pd.read_csv('../data/stock-list.csv', delimiter=';')

In [4]:
stock_list.sample(5)

,No,Kode,Nama Perusahaan,Tanggal Pencatatan,Saham,Papan Pencatatan
359,360,PLIN,Plaza Indonesia Realty Tbk.,15/06/1992,3550000000,Pemantauan Khusus
838,839,JATI,Informasi Teknologi Indonesia,08/05/2023,3262520106,Pengembangan
174,175,ETWA,Eterindo Wahanatama Tbk,16/05/1997,4668671400,Pemantauan Khusus
3,4,ABMM,ABM Investama Tbk.,06/12/2011,2753165000,Utama
880,881,SURI,Maja Agung Latexindo Tbk.,07/12/2023,6334375000,Pengembangan


# Scrape financial report data


In [5]:
api_key = os.getenv("STOCKBIT_API_KEY", "")

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    'Authorization': f'Bearer {api_key}'
}

In [7]:
symbols = stock_list['Kode'].tolist()

statement_types = [
    {"annual": 2},
]

report_types = [
    {"income_statement": 1},
    {"balance_sheet": 2},
    {"cash_flow_statement": 3},
]

In [8]:
# Loop through each stock symbol
for symbol in symbols:
    for statement in statement_types:
        # Create directory for the symbol if it doesn't exist
        os.makedirs(f'../data/financials/{symbol}/{list(statement.keys())[0]}', exist_ok=True)

        for report in report_types:
            statement_type = list(statement.values())[0]
            report_type = list(report.values())[0]

            url = f'https://exodus.stockbit.com/findata-view/company/financial?symbol={symbol}&data_type=1&report_type={report_type}&statement_type={statement_type}'

            print(f"Fetching data for {symbol} from URL: {url}")

            try:
                req = urllib.request.Request(url, headers=headers)
                res = urllib.request.urlopen(req)

                parsed_res = json.loads(res.read().decode('utf-8'))

                html_data = BeautifulSoup(parsed_res["data"]["html_report"], 'html.parser')

                # Save the HTML content to a file
                with open(f'../data/financials/{symbol}/{list(statement.keys())[0]}/{list(report.keys())[0]}.html', 'w', encoding='utf-8') as file:
                    file.write(str(html_data))

                spans = html_data.find_all('span')

                # Iterate through the found spans
                for span in spans:
                    # Check if the 'data-lang-0-full' attribute exists
                    if 'data-lang-0-full' in span.attrs:
                        new_text = span['data-lang-0-full']
                        # Replace the current text content of the span
                        span.string = new_text

                    # Unwrap the span to get the text content
                    span.unwrap()

                table_data = pd.read_html(StringIO(str(html_data)))

                for i, table_datum in enumerate(table_data):
                    # Transpose the table
                    table_transposed = table_datum.transpose()
                    # Get the first row as the header
                    table_transposed.columns = table_transposed.iloc[0]
                    # Include index as a column on first position
                    table_transposed.reset_index(names="Period", inplace=True)
                    # Remove the first row
                    table_transposed = table_transposed[1:]
                    # Remove all &nbsp characters from column names
                    table_transposed.columns = table_transposed.columns.str.replace('\\&nbsp', '', regex=True)
                    # Replace all '-' with NaN
                    table_transposed.replace('-', pd.NA, inplace=True)
                    # Save table to a CSV file
                    table_transposed.to_csv(f'../data/financials/{symbol}/{list(statement.keys())[0]}/{list(report.keys())[0]}_{i}.csv', index=False)

                    print(f"Data for {symbol} saved successfully.")
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")

Fetching data for AALI from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=AALI&data_type=1&report_type=1&statement_type=2
Data for AALI saved successfully.
Data for AALI saved successfully.
Fetching data for AALI from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=AALI&data_type=1&report_type=2&statement_type=2


C:\Users\Rulim\AppData\Local\Temp\ipykernel_18992\1640642437.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_transposed.replace('-', pd.NA, inplace=True)


Data for AALI saved successfully.
Data for AALI saved successfully.
Data for AALI saved successfully.
Data for AALI saved successfully.
Fetching data for AALI from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=AALI&data_type=1&report_type=3&statement_type=2
Data for AALI saved successfully.
Data for AALI saved successfully.
Fetching data for ABBA from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=ABBA&data_type=1&report_type=1&statement_type=2
Data for ABBA saved successfully.
Data for ABBA saved successfully.
Fetching data for ABBA from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=ABBA&data_type=1&report_type=2&statement_type=2


C:\Users\Rulim\AppData\Local\Temp\ipykernel_18992\1640642437.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_transposed.replace('-', pd.NA, inplace=True)


Data for ABBA saved successfully.
Data for ABBA saved successfully.
Data for ABBA saved successfully.
Data for ABBA saved successfully.
Fetching data for ABBA from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=ABBA&data_type=1&report_type=3&statement_type=2
Data for ABBA saved successfully.
Data for ABBA saved successfully.
Fetching data for ABDA from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=ABDA&data_type=1&report_type=1&statement_type=2


...

Data for EURO saved successfully.
Data for EURO saved successfully.
Data for EURO saved successfully.
Data for EURO saved successfully.
Fetching data for EURO from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=EURO&data_type=1&report_type=3&statement_type=2
Data for EURO saved successfully.
Data for EURO saved successfully.
Fetching data for KLIN from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=KLIN&data_type=1&report_type=1&statement_type=2
Data for KLIN saved successfully.
Data for KLIN saved successfully.
Fetching data for KLIN from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=KLIN&data_type=1&report_type=2&statement_type=2


C:\Users\Rulim\AppData\Local\Temp\ipykernel_18992\1640642437.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_transposed.replace('-', pd.NA, inplace=True)


Data for KLIN saved successfully.
Data for KLIN saved successfully.
Data for KLIN saved successfully.
Data for KLIN saved successfully.
Fetching data for KLIN from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=KLIN&data_type=1&report_type=3&statement_type=2
Data for KLIN saved successfully.
Data for KLIN saved successfully.
Fetching data for TOOL from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=TOOL&data_type=1&report_type=1&statement_type=2
Data for TOOL saved successfully.
Data for TOOL saved successfully.
Fetching data for TOOL from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=TOOL&data_type=1&report_type=2&statement_type=2


C:\Users\Rulim\AppData\Local\Temp\ipykernel_18992\1640642437.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table_transposed.replace('-', pd.NA, inplace=True)


Data for TOOL saved successfully.
Data for TOOL saved successfully.
Data for TOOL saved successfully.
Data for TOOL saved successfully.
Fetching data for TOOL from URL: https://exodus.stockbit.com/findata-view/company/financial?symbol=TOOL&data_type=1&report_type=3&statement_type=2
Data for TOOL saved successfully.
Data for TOOL saved successfully.
